In [3]:
import torch
from task_vectors import TaskVector
# Create the task vector
task_vector = TaskVector("llama-3-8b.pt", "llama-3-8b-inst.pt")

/home/richard/workspace/task_vectors/src/task_vectors.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_state_dict = torch.load(pretrained_checkpoint).state_di

In [4]:
torch.save({
    "model_state_dict": task_vector
}, "llama-3-8b-inst-task-vector.pt")

In [5]:
from transformers import AutoModelForCausalLM
sapie_inst = AutoModelForCausalLM.from_pretrained("4yo1/llama3-pre1-pre2-inst3-ds-lora3", cache_dir="cache")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
import torch
torch.save(sapie_inst, "sapie_inst3.pt")

In [7]:
sapie_inst

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(149465, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [8]:
import torch
loaded_data = torch.load("llama-3-8b-inst-task-vector.pt")
task_vector = loaded_data["model_state_dict"]

/tmp/ipykernel_902024/282690055.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_data = torch.load("llama-3-8b-inst-task-vector.pt")


In [9]:
sapie_inst3_inst = task_vector.apply_to("sapie_inst3.pt")

/home/richard/workspace/task_vectors/src/task_vectors.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_model = torch.load(pretrained_checkpoint)


In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# Load the tokenizer and model
# model_name = "llama-3-model"  # Replace with actual model identifier if available
tokenizer = AutoTokenizer.from_pretrained("4yo1/llama3-pre1-pre2-inst3-ds-lora3")
tokenizer.pad_token = tokenizer.eos_token
model = sapie_inst3_inst.to("cuda")

In [14]:

# Define a prompt
prompt =  '입력된 영어 문장을 한국어로 번역하여 출력하세요. 번역만 출력하세요.\n\nYoduk Story is an epic musical by North Korean defector Chung Sung-san who exposed human rights abuses in North Korean political prison camps based on true stories.\n\n'
# prompt = "Once upon a time in a land far, far away,"
# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate text
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=200,  # Maximum length of generated text
    num_beams=5,  # Number of beams for beam search
    no_repeat_ngram_size=2,  # Avoid repeating n-grams
    early_stopping=True
)
# Decode the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated Text:
입력된 영어 문장을 한국어로 번역하여 출력하세요. 번역만 출력하세요.

Yoduk Story is an epic musical by North Korean defector Chung Sung-san who exposed human rights abuses in North Korean political prison camps based on true stories.

yoduk 스토리는 북한 탈출자 정성산씨의 실화를 담아낸 초대형 음악극이에요, 남북이라도 합칠 수 있으면 좋겠다고 생각합니다.？」

Yoduk story에 대해서만 말씀드리고자하는건 바로 이야기를해주실 분들께서 알려주셨던 내용으로써 조금씩 설명해드릴 것이며,그렇게부터 출처가 된
